In [ ]:
# RDKIt libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs
from rdkit import DataStructs

In [ ]:
# Calculate similarity between training and validation datasets
def calc_mol_sim(train_mols, ex_mols):
    ress = []
    for i  in range(0,len(train_mols)):
        train_smi = Chem.MolFromSmiles(train_mols.iloc[i]['canonical_smiles'])
        train_smi.SetProp("_Name", train_mols.iloc[i]['molecule_chembl_id'])
        for n in range (0, len(train_mols)):
            ex_smi = Chem.MolFromSmiles(ex_mols.iloc[n]['canonical_smiles'])
            ex_smi.SetProp("_Name", ex_mols.iloc[n]['molecule_chembl_id'])
            fitMolTITLE = ex_smi.GetProp("_Name")
            refMolTITLE = train_smi.GetProp("_Name")
            # Atom Pairs
            try:
                refATMPfp = Pairs.GetAtomPairFingerprintAsBitVect(train_smi)
                fitATMPfp = Pairs.GetAtomPairFingerprintAsBitVect(ex_smi)
                atmpFING_score = float(round(DataStructs.DiceSimilarity(refATMPfp,
                                                                fitATMPfp), 3))
            except:
                atmpFING_score = 0.000
            ress.append([refMolTITLE, fitMolTITLE, atmpFING_score])
    df_ress = pd.DataFrame(ress, columns=["Ex_mol", "Train_mol",'APfp_score'])
    df_stats = df_ress.groupby(by='Train_mol').mean().join(
                        df_ress.groupby(by='Train_mol').std(),
                                lsuffix='_mean', rsuffix='_std').join(
                        df_ress.groupby(by='Train_mol').max(),rsuffix='_max').join(
                        df_ress.groupby(by='Train_mol').min(),
                                lsuffix='_max', rsuffix='_min')
#    df_prob_label_stats =  df_prob_label.set_index('Molecule ChEMBL ID').join(df_stats)
#    df_prob_label_stats.to_csv('csv_res-test-train_analyze-SIMILARITY-prob-binary_' + str(num) + '-' + isoform_df.name.replace('/', '-') + '_PCC_095.csv')
#    df_stats.to_csv('csv_res-test-train_analyze-SIMILARITY-prob-binary_' + str(num) + '-' + isoform_df.name.replace('/', '-') + '_PCC_095.csv')
    return df_stats

In [ ]:
# Calculate similarity 
train_mols = df_morgan2.loc[X_train.index][['canonical_smiles','molecule_chembl_id']]
ex_mols = dfchem.loc[Xex.index][['canonical_smiles','molecule_chembl_id']]
df_stats = calc_mol_sim(ex_mols, train_mols)
df_stats_ex = calc_mol_sim(ex_mols, ex_mols)
df_stats_tt = calc_mol_sim(train_mols, train_mols)
df_stats.to_csv('csv_res-train-ex_analyze-SIMILARITY-prob-binary_'   + 'PPARd_ex'+'_PCC_095.csv')
df_stats_ex.to_csv('csv_res-ex-ex_analyze-SIMILARITY-prob-binary_'  + 'PPARd_ex' +'_PCC_095.csv')
df_stats_tt.to_csv('csv_res-train-train_analyze-SIMILARITY-prob-binary_'  + 'PPARd_ex' +'_PCC_095.csv')